In [ ]:
#https://data.mendeley.com/datasets/5y9wdsg2zt/1 (data)
import os, shutil

In [ ]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_pos_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/Positive'
original_dataset_neg_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/Negative'

# The directory where we will
# store our smaller dataset
base_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/CNN_CPU'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training posive pictures
train_pos_dir = os.path.join(train_dir, 'pos')
os.mkdir(train_pos_dir)

# Directory with our training negative pictures
train_neg_dir = os.path.join(train_dir, 'neg')
os.mkdir(train_neg_dir)

# Directory with our validation positive pictures
validation_pos_dir = os.path.join(validation_dir, 'pos')
os.mkdir(validation_pos_dir)

# Directory with our validation negative pictures
validation_neg_dir = os.path.join(validation_dir, 'neg')
os.mkdir(validation_neg_dir)

# Directory with our validation positive pictures
test_pos_dir = os.path.join(test_dir, 'pos')
os.mkdir(test_pos_dir)

# Directory with our validation negative pictures
test_neg_dir = os.path.join(test_dir, 'neg')
os.mkdir(test_neg_dir)

In [ ]:
# change file name such as 10000_1.jpg to 10000.jpg
for f in os.listdir(original_dataset_pos_dir):
    fname_split = f.split("_")
    if len(fname_split)!=1:
        src = os.path.join(original_dataset_pos_dir,f)
        dst = os.path.join(original_dataset_pos_dir,fname_split[0]+'.jpg')
        os.rename(src, dst) 

for f in os.listdir(original_dataset_neg_dir):
    fname_split = f.split("_")
    if len(fname_split)!=1:
        src = os.path.join(original_dataset_neg_dir,f)
        dst = os.path.join(original_dataset_neg_dir,fname_split[0]+'.jpg')
        os.rename(src, dst) 

# Note: may need to shuffle the images then copy to train, test, validation

In [ ]:
# Copy first 14000 images to train
fnames = ['{0:05}.jpg'.format(i) for i in range(1,14001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(train_pos_dir, fname)
    dst_neg = os.path.join(train_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)

# Copy first 4000 images to validation
fnames = ['{0:05}.jpg'.format(i) for i in range(14001,18001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(validation_pos_dir, fname)
    dst_neg = os.path.join(validation_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)

    
# Copy first 2000 images to test
fnames = ['{0:05}.jpg'.format(i) for i in range(18001,20001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(test_pos_dir, fname)
    dst_neg = os.path.join(test_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)
    
  

In [ ]:
print('total training pos images:', len(os.listdir(train_pos_dir)))
print('total training neg images:', len(os.listdir(train_neg_dir)))

In [ ]:
print('total test pos images:', len(os.listdir(test_pos_dir)))
print('total test neg images:', len(os.listdir(test_neg_dir)))

In [ ]:
print('total validation pos images:', len(os.listdir(validation_pos_dir)))
print('total validation neg images:', len(os.listdir(validation_neg_dir)))

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(227, 227, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(227, 227),
        batch_size=140,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(227, 227),
        batch_size=100,
        class_mode='binary')

In [ ]:
train_dir

In [ ]:
validation_dir

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=40)